## 雜訊抑制

### 1. 預處理：使用濾波器去除低頻和高頻雜訊

- 低通濾波器（Low-Pass Filter）：去除高頻噪音，保留低頻信號部分，這對於高頻噪音較多的信號很有效。
- 高通濾波器（High-Pass Filter）：去除低頻噪音，保留高頻信號部分，適合去除低頻背景噪音。
- 帶通濾波器（Band-Pass Filter）：根據你的信號特性，使用帶通濾波器僅保留信號的特定頻率範圍，去除非目標頻段的噪音。

In [ ]:
from scipy.signal import butter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# 設置濾波器參數
lowcut = 300.0
highcut = 3000.0
fs = 44100

# 應用濾波器
filtered_signal = bandpass_filter(signal, lowcut, highcut, fs)


### 2. 使用移動平均或其他平滑技術

- 移動平均（Moving Average）：使用移動平均技術對信號進行平滑處理，可以減少雜訊的影響，保留主要波形特徵。
- 指數平滑（Exponential Smoothing）：適合於信號波動較大的情況，能更快響應信號變化。

In [ ]:
def moving_average(signal, window_size):
    return np.convolve(signal, np.ones(window_size)/window_size, mode='same')

# 應用移動平均
smoothed_signal = moving_average(filtered_signal, window_size=5)


### 3. 基於頻率域的處理
- 短時傅立葉變換（STFT）：將信號轉換到頻率域進行分析，去除特定頻段的噪音。STFT能幫助識別並分離不同頻率分量。
- 小波變換（Wavelet Transform）：使用小波變換來進行多尺度分析，這種方法在去除噪音和提取特定信號方面具有良好的性能。

In [ ]:
import librosa
import librosa.display

# 計算STFT
D = librosa.stft(filtered_signal)

# 將頻譜中的噪音部分置零
threshold = np.median(np.abs(D))
D[np.abs(D) < threshold] = 0

# 重建信號
denoised_signal = librosa.istft(D)


#### 4. 使用自適應濾波器（Adaptive Filtering）
- 自適應濾波器可以根據輸入信號的特性動態調整其濾波器係數，是去除噪音的有效方法之一。常用的方法包括LMS（Least Mean Squares）濾波器。

In [ ]:
from scipy.signal import lfilter

def adaptive_filter(noisy_signal, desired_signal, mu=0.01, n=32):
    w = np.zeros(n)
    y = np.zeros(len(noisy_signal))
    e = np.zeros(len(noisy_signal))
    for i in range(n, len(noisy_signal)):
        x = noisy_signal[i:i-n:-1]
        y[i] = np.dot(w, x)
        e[i] = desired_signal[i] - y[i]
        w = w + 2 * mu * e[i] * x
    return e

# 假設desired_signal是理想信號
denoised_signal = adaptive_filter(noisy_signal=filtered_signal, desired_signal=desired_signal)


#### 5. 採用機器學習和深度學習方法
- 降噪自動編碼器（Denoising Autoencoder）：使用自動編碼器模型學習如何將噪音信號轉換為無噪音的信號。
- 深度卷積神經網絡（CNN）：訓練CNN模型來識別和去除噪音。

1. 設計CNN架構<br>
CNN去噪器的基本結構通常包括以下幾個主要部分：<br>
輸入層：接受噪音音頻信號或其頻譜表示（如梅爾頻譜、STFT頻譜等）。<br>
卷積層：提取信號中的局部特徵，識別噪音和信號的區別。<br>
池化層（可選）：進行降維，保留重要特徵，同時減少計算量。<br>
上採樣層：在使用池化層的情況下，進行上採樣以恢復信號的原始大小。<br>
輸出層：生成去噪後的音頻信號或其頻譜。<br>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Dense, Flatten, Reshape

model = Sequential()

# Encoder
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same', input_shape=(input_length, 1)))
model.add(MaxPooling1D(pool_size=2, padding='same'))

model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))

# Decoder
model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))

model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))

# Output Layer
model.add(Conv1D(1, kernel_size=3, activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='mean_squared_error')


2. 準備訓練數據
- 噪音數據集：需要準備一組含有噪音的音頻數據，以及相應的無噪音版本作為標籤。這些數據可以是真實的錄音，也可以是合成的。<br>
- 數據預處理：對音頻信號進行標準化、分幀處理，或者將其轉換為頻譜表示（如STFT或梅爾頻譜）以作為CNN的輸入。<br>

In [ ]:
import librosa
import numpy as np

def prepare_data(noisy_signals, clean_signals):
    X = []
    Y = []
    
    for noisy, clean in zip(noisy_signals, clean_signals):
        # 可以使用 librosa 提取特徵，如 STFT 或 MFCC
        noisy_stft = librosa.stft(noisy)
        clean_stft = librosa.stft(clean)
        
        X.append(np.abs(noisy_stft))
        Y.append(np.abs(clean_stft))
    
    X = np.array(X).reshape(len(X), X[0].shape[0], X[0].shape[1], 1)
    Y = np.array(Y).reshape(len(Y), Y[0].shape[0], Y[0].shape[1], 1)
    
    return X, Y

X_train, Y_train = prepare_data(noisy_signals_train, clean_signals_train)


3. 訓練模型
- 損失函數：通常使用均方誤差（MSE）作為損失函數來度量去噪效果的好壞。
- 優化器：使用 Adam 或其他適合的優化器進行模型訓練。



In [4]:
model.fit(X_train, Y_train, epochs=50, batch_size=64, validation_split=0.1)

NameError: name 'model' is not defined

4. 評估模型
- 使用測試數據進行評估：對於未見過的噪音信號，檢查模型的去噪效果。
- 指標：使用信噪比（SNR）、均方誤差（MSE）、感知評估指標（PESQ）等來評估去噪效果。

In [5]:

loss = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")

# Example to denoise
predicted_clean_signal = model.predict(noisy_signal)



NameError: name 'model' is not defined

In [ ]:
denoised_signal = model.predict(new_noisy_signal)

# 假設已經有 denoised_stft 由 model 預測得到
denoised_signal = librosa.istft(denoised_stft)

#### 6. 根據先驗知識進行手動分析和剪切
當自動方法難以有效去除噪音時，可以依賴先驗知識（例如音樂的特定頻率範圍或語音信號的特徵）來手動切割和選擇信號部分。